# Mauer Cory
# CS 614 Assignment 2: Recommender System
## Stock Website Reccomender

## Pitch: 
In a previous project I develop a Stock viewing website (https://mauer-stocks.azurewebsites.net/), and in this project I implemented a reccomender system that can reccomend stocks for users. The goal of this system is to improve the user experience while using the site and allow users to easily find new stocks. 

## Data source:
Since I wanted to develop a system that worked well with my website I was unable to leverage an existing data set and instead leveraged the favorites data base used by my website in conjuction with this stock listing website (https://companiesmarketcap.com/all-categories/). My data set was created by creating various users with randomly selected stocks accross various categories such as 
* Tech
* Retail 
* Realestate
* Finance
* Defense
* Health
As well as some mixed category users.  The idea behind the creation of this data set is that I expect many users to focus primarily on specific types of stocks, and wanted a system to would reccomend similar stocks to users. 

## Model and data justification:
For model selection I chose to go with collaborative filtering via matrix factorization approach using Non-negative matrix factorization (sklearn.decomposition.NMF). Since my data set was focused on a binary recomender (user has/has not favorited) and I wanted a highly performant system that could be updated everytime a new user is added, or stock is favorited this seemed like an optimal choice. Since I am currently using a fairly small data set (24 users and 253 stocks) spread across 7 independant types, I decided to perform factorizations using 10 features

## Commented examples:
The input of the model uses a sparse matrix built from users (rows) and their favorited stocks (columns). 

In [38]:
import os
import pandas as pd
from colab_filter import get_users, get_stocks, build_truth_matrix

truth_matrix = build_truth_matrix(get_users(), get_stocks())
pd.options.display.width = 150
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)

## example of the sparse matrix that includes all user favorites
print(truth_matrix)

             AAP  AAPL  ABR  ABT  ACI  ADBE  ADSK  AJRD  ALSN  AMC  ...  WDAY  WELL  WFC  WMT  WPM  WWD  XOM   ZG   ZM  ZUMZ
Bobby        0.0   0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0  1.0  ...   0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0
Nick2        0.0   0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0  0.0  ...   0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0
automotive   0.0   0.0  0.0  0.0  0.0   0.0   0.0   0.0   1.0  0.0  ...   0.0   0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0
automotive2  0.0   0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0  0.0  ...   0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0
commodity    0.0   0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0  0.0  ...   0.0   0.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0   0.0
...          ...   ...  ...  ...  ...   ...   ...   ...   ...  ...  ...   ...   ...  ...  ...  ...  ...  ...  ...  ...   ...
tech         0.0   1.0  0.0  0.0  0.0   1.0   0.0   0.0   0.0  0.0  ...   1.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0


In [39]:
# example of the factored user matrix with each users trained weights
from colab_filter import train_weights

number_of_features = 10 # number of features to be factorized
user_weights, stock_weights = train_weights(truth_matrix, number_of_features)
user_weights = pd.DataFrame(user_weights, index=get_users())
print(user_weights)

                    0         1    2         3    4         5         6         7         8         9
Bobby        0.000000  0.000000  0.0  0.000000  0.0  0.000000  0.000000  0.000000  0.000000  0.149416
Nick2        0.000000  0.000000  0.0  0.000000  0.0  0.127709  0.000000  0.000000  0.000000  0.000000
automotive   0.000000  2.520472  0.0  0.000000  0.0  0.000000  0.000000  0.157964  0.000283  0.000000
automotive2  0.000000  1.072822  0.0  0.000000  0.0  0.000000  0.000000  0.000000  0.000000  0.000000
commodity    0.000000  0.000000  0.0  0.000000  0.0  0.000000  2.108668  0.000000  0.000000  0.000000
...               ...       ...  ...       ...  ...       ...       ...       ...       ...       ...
tech         1.356407  0.009710  0.0  0.000000  0.0  0.000000  0.000000  0.000000  0.000000  0.000000
tech2        0.802468  0.000000  0.0  0.009709  0.0  0.000000  0.000000  0.000000  0.130352  0.000000
tech3        0.858544  0.000000  0.0  0.005589  0.0  0.000000  0.000000  0.000000 

In [41]:
# example of the factored stock matrix with each stocks trained weights
stock_weights = pd.DataFrame(stock_weights, columns=get_stocks())
pd.set_option('display.max_columns', 10)
print(stock_weights)

        AAP      AAPL       ABR       ABT       ACI  ...       WWD       XOM        ZG        ZM      ZUMZ
0  0.000000  0.767507  0.000000  0.000000  0.000000  ...  0.000000  0.225630  0.161137  0.183134  0.000000
1  0.000000  0.017028  0.000000  0.000000  0.000000  ...  0.000000  0.016281  0.000000  0.000000  0.000000
2  0.000000  0.000000  0.000002  0.000000  0.000000  ...  0.000000  0.000000  0.000000  0.000000  0.000000
3  0.280755  0.001868  0.000000  0.000000  0.280755  ...  0.000000  0.014157  0.000000  0.000000  0.381354
4  0.000000  0.011281  0.000000  0.321394  0.000000  ...  0.000000  0.028398  0.000000  0.000000  0.001271
5  0.000000  0.053300  0.000000  0.000000  0.000000  ...  0.384726  0.051505  0.000000  0.000000  0.000000
6  0.000000  0.077029  0.000000  0.000000  0.000000  ...  0.000000  0.570257  0.000000  0.000000  0.000000
7  0.000000  0.061634  0.000000  0.000000  0.000000  ...  0.000000  0.016952  0.000000  0.000000  0.000000
8  0.000000  0.009233  0.428834  0.00

Note that we are able to factor our truth matrix down to 2,530 parameters and 240 parameters down from the prediction matrix size of 6,0072 parameters. This allows us to calculate user favorites by multiplying the users row with the stock matrix which results in a 1x240 matrix(array) which can easily be sorted to find the users highest reccomended stocks. 

Note: That you can kind of see how the factorized matrices are gouping stocks toghether, for example AAPL (Apple) has high rating for feature 0, similar to the tech stock users.  

## Testing:
Provide a confusion matrix with one or more metrics and comment the results.
Code and instructions to run it:
Provide a link to the code and any required instructions to run it. Please include some testing examples so we can quickly experience what you experienced with the model.
### I agree to sharing this assignment with other students in the course after grading has occurred. 